In [0]:
import torch
from torch.autograd import Variable

In [0]:
class MyReLU(torch.autograd.Function):

    @staticmethod
    def forward(self, input_):
        # 在forward中，需要定义MyReLU这个运算的forward计算过程
        # 同时可以保存任何在后向传播中需要使用的变量值
        self.save_for_backward(input_)         # 将输入保存起来，在backward时使用
        output = input_.clamp(min=0)               # relu就是截断负数，让所有负数等于0
        return output

    @staticmethod
    def backward(self, grad_output):
        # 根据BP算法的推导（链式法则），dloss / dx = (dloss / doutput) * (doutput / dx)
        # dloss / doutput就是输入的参数grad_output、
        # 因此只需求relu的导数，在乘以grad_outpu    
        input_, = self.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input_ < 0] = 0                # 上诉计算的结果就是左式。即ReLU在反向传播中可以看做一个通道选择函数，所有未达到阈值（激活值<0）的单元的梯度都为0
        return grad_input

In [0]:
my_relu = MyReLU.apply

In [4]:
input1 = torch.tensor([-3.0, -1.5,  0.0,  1.5,  3.0],requires_grad=True);input1

tensor([-3.0000, -1.5000,  0.0000,  1.5000,  3.0000], requires_grad=True)

In [5]:
input2 = torch.tensor([-3.0, -1.5,  0.0,  1.5,  3.0],requires_grad=True);input2

tensor([-3.0000, -1.5000,  0.0000,  1.5000,  3.0000], requires_grad=True)

In [6]:
# 自定义实现 relu， forward,backward
my_res = my_relu(input1);my_res

tensor([0.0000, 0.0000, 0.0000, 1.5000, 3.0000], grad_fn=<MyReLUBackward>)

In [7]:
# pytorch 实现
import torch.nn as nn
relu = nn.ReLU()
py_res = relu(input2);py_res

tensor([0.0000, 0.0000, 0.0000, 1.5000, 3.0000], grad_fn=<ReluBackward0>)

In [8]:
my_loss = torch.sum(my_res);my_loss

tensor(4.5000, grad_fn=<SumBackward0>)

In [9]:
py_loss = torch.sum(py_res);py_loss

tensor(4.5000, grad_fn=<SumBackward0>)

In [0]:
my_loss.backward()

In [0]:
py_loss.backward()

In [12]:
input1.grad

tensor([0., 0., 1., 1., 1.])

In [13]:
input2.grad

tensor([0., 0., 0., 1., 1.])

In [0]:
# 自定义function 继承torch.nn.function，要实现 __init__, forward, backwards方法。
# https://zhuanlan.zhihu.com/p/27783097

In [16]:
from torch.autograd import gradcheck

# gradchek takes a tuple of tensor as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.

input_ = Variable(torch.tensor([-2.,-1.,2.]), requires_grad=True)
test = gradcheck(MyReLU.apply, input_, eps=1e-6, atol=1e-4)

/usr/local/lib/python3.6/dist-packages/torch/autograd/gradcheck.py:239: UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 
  'At least one of the inputs that requires gradient '


RuntimeError: ignored

### You don’t need to define the backward function, of you use just PyTorch code. Once you need another library, autograd cannot create the backward pass for you.
https://discuss.pytorch.org/t/now-that-pytorch-can-autograd-why-we-still-need-to-define-backward-when-defining-customer-function/16200

#### 纯pytorch代码不需要自定义backwards方法，pytorch会自己完成，但如果计算过程中调用外部非pytorch库就要自己实现backwards方法。

In [17]:
ip = torch.tensor([-2., -1., 0., 1., 2.], requires_grad=True);ip

tensor([-2., -1.,  0.,  1.,  2.], requires_grad=True)

In [20]:
loss = torch.sum(ip[ip>0]);loss

tensor(3., grad_fn=<SumBackward0>)

In [0]:
loss.backward()

In [25]:
ip.grad

tensor([0., 0., 0., 1., 1.])